# Q3: AI 세율을 정책적으로 적용했을 때의 거시경제적 효과?

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from linearmodels.panel import PanelOLS
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge

file_path = "../data/master_data_by_category.csv"
df = pd.read_csv(file_path)

df = df.pivot(index=['Country', 'Year'], columns='category', values='Value')

df = df[['GDP', 'Corporate Tax', 'Patent Publications', 'GERD', 'WIPO Tax', 'Unemployment Rate']].dropna()

## 1. 반사실 분석 (Counterfactual Analysis)

In [ ]:
# 가상의 AI 세율 설정 (기존의 Corporate Tax & WIPO Tax 기반)
df['AI Tax Simulated'] = df['WIPO Tax'] * 0.5 + df['Corporate Tax'] * 0.5

# 패널 회귀 분석(AI 세율이 경제 성장에 미치는 영향)
X = df[['AI Tax Simulated', 'Corporate Tax', 'WIPO Tax']]
y = df['GDP']
X = sm.add_constant(X)

# 패널 OLS 회귀 분석
model = sm.OLS(y, X).fit()
print("\n Counterfactual AI Tax Regression Results")
print(model.summary())

## 2. 합성 통제법 (Synthetic Control Method)

In [ ]:
# PCA를 이용하여 AI 세율이 적용된 가상의 국가 생성
features = df[['Corporate Tax', 'WIPO Tax', 'Patent Publications', 'GERD']]
pca = PCA(n_components=1)
df['Synthetic AI Tax'] = pca.fit_transform(features)

# 합성 통제 국가의 경제 성장 예측(Ridge 회귀 활용)
X_syn = df[['Synthetic AI Tax', 'Corporate Tax', 'WIPO Tax']]
y_syn = df['GDP']
X_syn = sm.add_constant(X_syn)

synthetic_model = sm.OLS(y_syn, X_syn).fit()
print("\n Synthetic Control Method Regression Results")
print(synthetic_model.summary())

## 3. 거시경제 시뮬레이션 (Macroeconomic Simulation)

In [ ]:
def economic_growth_simulation(tax_rate, beta0, beta1, beta2):
    return beta0 + beta1 * tax_rate - beta2 * tax_rate**2

# 최적화 함수
def optimal_tax_growth(tax_rate, beta0, beta1, beta2):
    return -economic_growth_simulation(tax_rate, beta0, beta1, beta2)

In [ ]:
# 회귀 계수 추출
beta0, beta1, beta2 = model.params[0], model.params[1], model.params[1] / 10

# 최적 AI 세율 찾기
res = minimize(optimal_tax_growth, x0=0.2, args=(beta0, beta1, beta2), bounds=[(0, 1)])
optimal_tax = res.x[0]

print(f"\n 최적 AI 세율 (거시경제 시뮬레이션 결과): {optimal_tax:.2%}")

In [ ]:
tax_rates = np.linspace(0, 1, 100)
growth_rates = economic_growth_simulation(tax_rates, beta0, beta1, beta2)

plt.figure(figsize=(8, 5))
plt.plot(tax_rates, growth_rates, label="GDP Growth vs. AI Tax")
plt.axvline(optimal_tax, color='r', linestyle='--', label=f'Optimal AI Tax: {optimal_tax:.2%}')
plt.xlabel("AI Tax Rate")
plt.ylabel("GDP Growth Rate (%)")
plt.legend()
plt.title("Optimal AI Tax Rate(using Macroeconomic Simulation)")
plt.show()